<a href="https://colab.research.google.com/github/pavanreddyml/FLAIRS-38-Material/blob/main/notebooks/02%20-%20Tracking_And_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands On Adversarial Attacks

# Setup

In [ ]:
!pip install adversarial-lab

In [ ]:
!git clone https://github.com/pavanreddyml/FLAIRS-38-Material.git

In [ ]:
from adversarial_lab.analytics import *
from adversarial_lab.db import SqlliteDB
from adversarial_lab.core.optimizers import PGD
from adversarial_lab.core.losses import CategoricalCrossEntropy
from adversarial_lab.attacker.whitebox import WhiteBoxMisclassification
from adversarial_lab.core.noise_generators import AdditiveNoiseGenerator
from adversarial_lab.core.preprocessing import PreprocessingFromFunction
from adversarial_lab.core.constraints import POClip, PONoisedSampleBounding
from adversarial_lab.callbacks import EarlyStopping
from adversarial_lab.arsenal.whitebox import *

from adversarial_lab.utils import Plotting, PlottingFromDB, VideoPlotter

import os
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [ ]:
MODEL = "InceptionV3"     # Supported models: InceptionV3, ResNet50, MobileNetV2, Digits
IMAGES_DIR = os.path.join("FLAIRS-38-Material", "data", "animals")     # Directory containing the images
MODEL_DIR = os.path.join("FLAIRS-38-Material", "models")               # Directory containing the model

In [ ]:
if MODEL == "InceptionV3":
    from tensorflow.keras.applications import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
    model = InceptionV3(weights='imagenet')
    input_shape = (299, 299, 3)
elif MODEL == "ResNet50":
    from tensorflow.keras.applications import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
    model = ResNet50(weights='imagenet')
    input_shape = (224, 224, 3)
elif MODEL == "MobileNetV2":
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    model = MobileNetV2(weights='imagenet')
    input_shape = (224, 224, 3)
else:
  raise ValueError(f"Unsupported Model: {MODEL}")

In [ ]:
def preprocess(sample, *args, **kwargs):
    input_sample = tf.cast(sample, dtype=tf.float32)
    if len(input_sample.shape) == 2:
        input_sample = tf.expand_dims(input_sample, axis=-1)
        input_sample = tf.image.grayscale_to_rgb(input_sample)

    elif len(input_sample.shape) == 3 and input_sample.shape[-1] == 1:
        input_sample = tf.image.grayscale_to_rgb(input_sample)

    input_tensor = tf.convert_to_tensor(input_sample, dtype=tf.float32)
    resized_image = tf.image.resize(input_tensor, input_shape[:2])
    batch_image = tf.expand_dims(resized_image, axis=0)
    return preprocess_input(batch_image)

In [ ]:
def get_image_array(image_path):
    image = Image.open(image_path)
    image_array = np.array(image)
    return image_array

# Initialize Tracking and Analytics

In [ ]:
db = SqlliteDB(db_path="whitebox.db")

def get_analytics(table_name):
  analytics = AdversarialAnalytics(
      db=db,
      trackers=[
          ImageTracker(),
          LossTracker(),
          NoiseStatisticsTracker(),
          NoiseTracker(),
          PredictionsTracker()
      ],
      table_name=table_name,
      force_create_table=True
  )
  return analytics

## BIM And PGD

In [ ]:
IMAGE = "panda.jpg"
EPSILON = 5
LEARNING_RATE = 1
TARGET_CLASS = 924
TARGET_CLASS_CONFIDENCE = 0.5
ON_ORIGINAL = True
EARLY_STOP = False
VERBOSE = 3
EPOCHS = 100

In [ ]:
image_path = os.path.join(IMAGES_DIR, IMAGE)
image_array = get_image_array(image_path)

In [ ]:
predictions = model.predict(preprocess(image_array), verbose=0)
print("Predicted class:", decode_predictions(predictions, top=1)[0][0][1])
print("Predicted class index:", np.argmax(predictions, axis=1)[0])
print("Predicted class probability:", np.max(predictions, axis=1)[0])

In [ ]:
attacker = ProjectedGradientDescentAttack(
    model=model,
    preprocessing_fn=PreprocessingFromFunction.create(preprocess),
    learning_rate=float(LEARNING_RATE),
    epsilon=float(EPSILON),
    binary=False,
    verbose=VERBOSE,
    callbacks=[EarlyStopping(trigger="misclassification", confidence=0.5)] if EARLY_STOP else None,
    analytics=get_analytics("BIM"),
)

noise, noise_meta = attacker.attack(
    sample=image_array,
    target_class=TARGET_CLASS,
    on_original=ON_ORIGINAL,
    epochs=EPOCHS
)

Plotting.plot_images_and_noise(image_array,
                               noise,
                               config={
                                   "title": f"BIM (Epsilon: {EPSILON}, Learning Rate: {LEARNING_RATE})",
                               })

In [ ]:
print(
    f"Epsilon: {EPSILON}\n"
    f"Learning Rate: {LEARNING_RATE}\n"
    f"Original class: {np.argmax(model.predict(preprocess(image_array), verbose=0), axis=1)[0]}\n"
    f"Target class: {TARGET_CLASS}\n"
    f"Adversarial class: {np.argmax(model.predict(preprocess(image_array + noise), verbose=0), axis=1)[0]}\n"
    f"Adversarial class confidence: {np.max(model.predict(preprocess(image_array + noise), verbose=0), axis=1)[0]}\n"
    f"Noise min: {np.min(noise)}\n"
    f"Noise max: {np.max(noise)}\n"
)

In [ ]:
plotter = PlottingFromDB(db, table_name="BIM")

In [ ]:
plotter.plot_losses()

In [ ]:
plotter.plot_predictions(idx=[388, 924])

In [ ]:
v_plotter = VideoPlotter(db, table_name="BIM")
v_plotter.make_video(save_path="BIM.mp4")

## Carlini Wagner Attack (C&W)

In [ ]:
IMAGE = "panda.jpg"
C=50,
KAPPA=0.5,
LEARNING_RATE = 1
TARGET_CLASS = 924
TARGET_CLASS_CONFIDENCE = 0.5
ON_ORIGINAL = True
EARLY_STOP = False
VERBOSE = 3
EPOCHS = 100

In [ ]:
image_path = os.path.join(IMAGES_DIR, IMAGE)
image_array = get_image_array(image_path)

In [ ]:
predictions = model.predict(preprocess(image_array), verbose=0)
print("Predicted class:", decode_predictions(predictions, top=1)[0][0][1])
print("Predicted class index:", np.argmax(predictions, axis=1)[0])
print("Predicted class probability:", np.max(predictions, axis=1)[0])

In [ ]:
attacker = CarliniWagnerAttack(
    model=model,
    preprocessing_fn=PreprocessingFromFunction.create(preprocess),
    learning_rate=float(LEARNING_RATE),
    C=C,
    kappa=KAPPA,
    binary=False,
    verbose=VERBOSE,
    callbacks=[EarlyStopping(trigger="misclassification", confidence=TARGET_CLASS_CONFIDENCE)] if EARLY_STOP else None,
    analytics=get_analytics("CW")
)

noise, noise_meta = attacker.attack(
    sample=image_array,
    target_class=TARGET_CLASS,
    on_original=ON_ORIGINAL, # This determines if the attack is performed on the original image or on the preprocessed one
    epochs=EPOCHS
)

Plotting.plot_images_and_noise(image_array,
                               noise,
                               config={
                                   "title": f"C&W Attack (C: {C}, Kappa: {KAPPA})",
                               })

In [ ]:
print(
    f"C: {C}\n"
    f"KAPPA: {KAPPA}\n"
    f"Original class: {np.argmax(model.predict(preprocess(image_array), verbose=0), axis=1)[0]}\n"
    f"Target class: {TARGET_CLASS}\n"
    f"Adversarial class: {np.argmax(model.predict(preprocess(image_array + noise), verbose=0), axis=1)[0]}\n"
    f"Adversarial class confidence: {np.max(model.predict(preprocess(image_array + noise), verbose=0), axis=1)[0]}\n"
    f"Noise min: {np.min(noise)}\n"
    f"Noise max: {np.max(noise)}\n"
)

In [ ]:
plotter = PlottingFromDB(db, table_name="BIM")

In [ ]:
plotter.plot_losses()

In [ ]:
plotter.plot_predictions(idx=[388, 924])

In [ ]:
v_plotter = VideoPlotter(db, table_name="BIM")
v_plotter.make_video(save_path="CW.mp4")

## Deepfool

In [ ]:
IMAGE = "panda.jpg"
OVERSHOOT = 0.1
OVERSHOOT_VALS = [0.1, 0.25, 0.5]
TARGET_CLASS = 924
TARGET_CLASS_CONFIDENCE = 0.3
ON_ORIGINAL = False                     # Deep Fool does not support on_original=True
EARLY_STOP = False
VERBOSE = 3
EPOCHS = 100
EFFICIENT_MODE = 20

In [ ]:
image_path = os.path.join(IMAGES_DIR, IMAGE)
image_array = get_image_array(image_path)

In [ ]:
predictions = model.predict(preprocess(image_array), verbose=0)
print("Predicted class:", decode_predictions(predictions, top=1)[0][0][1])
print("Predicted class index:", np.argmax(predictions, axis=1)[0])
print("Predicted class probability:", np.max(predictions, axis=1)[0])

In [ ]:
early_stopping = EarlyStopping(trigger="misclassification", confidence=0.2)
preprocessing_fn = PreprocessingFromFunction.create(preprocess)

attacker = DeepFoolAttack(
    model=model,
    preprocessing_fn=preprocessing_fn,
    epsilon=EPSILON,
    overshoot=OVERSHOOT,
    callbacks=[EarlyStopping(trigger="misclassification", confidence=TARGET_CLASS_CONFIDENCE)] if EARLY_STOP else None,
    analytics=get_analytics("DeepFool"),
    efficient_mode=EFFICIENT_MODE,
)

noise, noise_meta = attacker.attack(
    sample=image_array,
    target_class=924,
    on_original=ON_ORIGINAL,
    epochs=EPOCHS,
)

Plotting.plot_images_and_noise(preprocessing_fn.preprocess(image_array).numpy(),
                               noise,
                               config={
                                   "title": f"Deep Fool (Overshoot: {OVERSHOOT})",
                               })

In [ ]:
print(
    f"Overshoot: {OVERSHOOT}\n"
    f"Original class: {np.argmax(model.predict(preprocess(image_array), verbose=0), axis=1)[0]}\n"
    f"Target class: {TARGET_CLASS}\n"
    f"Adversarial class: {np.argmax(model.predict(preprocess(image_array) + noise, verbose=0), axis=1)[0]}\n"
    f"Adversarial class confidence: {np.max(model.predict(preprocess(image_array) + noise, verbose=0), axis=1)[0]}\n"
    f"Noise min: {np.min(noise)}\n"
    f"Noise max: {np.max(noise)}\n"
)

In [ ]:
plotter = PlottingFromDB(db, table_name="DeepFool")

In [ ]:
plotter.plot_losses()

In [ ]:
plotter.plot_predictions(idx=[388, 924])

In [ ]:
v_plotter = VideoPlotter(db, table_name="DeepFool")
v_plotter.make_video(save_path="DeepFool.mp4")

## Smooth Fool

In [ ]:
IMAGE = "panda.jpg"
OVERSHOOT = 0.1
SIGMA = 0.5
KERNEL_SIZE = 3
TARGET_CLASS = 924
TARGET_CLASS_CONFIDENCE = 0.3
ON_ORIGINAL = False                     # Smooth Fool does not support on_original=True
EARLY_STOP = False
VERBOSE = 3
EPOCHS = 100
EFFICIENT_MODE = 20

In [ ]:
image_path = os.path.join(IMAGES_DIR, IMAGE)
image_array = get_image_array(image_path)

In [ ]:
predictions = model.predict(preprocess(image_array), verbose=0)
print("Predicted class:", decode_predictions(predictions, top=1)[0][0][1])
print("Predicted class index:", np.argmax(predictions, axis=1)[0])
print("Predicted class probability:", np.max(predictions, axis=1)[0])

In [ ]:
early_stopping = EarlyStopping(trigger="misclassification", confidence=0.2)
preprocessing_fn = PreprocessingFromFunction.create(preprocess)

attacker = SmoothFoolAttack(
    model=model,
    preprocessing_fn=preprocessing_fn,
    epsilon=EPSILON,
    overshoot=OVERSHOOT,
    sigma=SIGMA,
    kernel_size=KERNEL_SIZE,
    callbacks=[EarlyStopping(trigger="misclassification", confidence=TARGET_CLASS_CONFIDENCE)] if EARLY_STOP else None,
    analytics=get_analytics("SmoothFool"),
    efficient_mode=EFFICIENT_MODE,
)

noise, noise_meta = attacker.attack(
    sample=image_array,
    target_class=924,
    epochs=EPOCHS,
)

Plotting.plot_images_and_noise(preprocessing_fn.preprocess(image_array).numpy(), noise)

In [ ]:
print(
    f"Overshoot: {OVERSHOOT}\n"
    f"Sigma: {SIGMA}\n"
    f"Kernel size: {KERNEL_SIZE}\n"
    f"Original class: {np.argmax(model.predict(preprocess(image_array), verbose=0), axis=1)[0]}\n"
    f"Target class: {TARGET_CLASS}\n"
    f"Adversarial class: {np.argmax(model.predict(preprocess(image_array) + noise, verbose=0), axis=1)[0]}\n"
    f"Adversarial class confidence: {np.max(model.predict(preprocess(image_array) + noise, verbose=0), axis=1)[0]}\n"
    f"Noise min: {np.min(noise)}\n"
    f"Noise max: {np.max(noise)}\n"
)

In [ ]:
plotter = PlottingFromDB(db, table_name="SmoothFool")

In [ ]:
plotter.plot_losses()

In [ ]:
plotter.plot_predictions(idx=[388, 924])

In [ ]:
v_plotter = VideoPlotter(db, table_name="SmoothFool")
v_plotter.make_video(save_path="SmoothFool.mp4")